# Observation of a charge-neutral muon polaron complex in antiferromagnetic Cr$_2$O$_3$
## Supplemental material: Convergence tests of the plane wave cutoff and k-point sampling

M. H. Dehn$^{1, 2, 3}$ J. K. Shenton$^{4,*}$ S. Holenstein$^{5,6}$ Q. N. Meier$^4$ D. J. Arseneau$^3$ D. L. Cortie$^{1, 2, 7, †}$ B. Hitti$^3$ A. C. Y. Fang$^{1, ‡}$ W. A. MacFarlane$^{2, 3, 7}$ R. M. L. McFadden$^{2, 7}$ G.
D. Morris$^3$ Z. Salman$^6$ H. Luetkens$^6$ N. A. Spaldin$^4$ M. Fechner$^{4, 8}$ and R. F. Kiefl$^{1, 2, 3}$


$^1$Department of Physics and Astronomy, University of British Columbia, Vancouver, BC V6T 1Z1, Canada    
$^2$Stewart Blusson Quantum Matter Institute, University of British Columbia, Vancouver, BC V6T 1Z4, Canada    
$^3$<span style="font-variant:small-caps;">Triumf</span>, Vancouver, BC V6T 2A3, Canada   
$^4$Department of Materials, ETH Zürich, CH-8093 Zürich, Switzerland   
$^5$Physik-Institut der Universität Zürich, CH-8057, Zürich, Switzerland   
$^6$Laboratory for Muon Spin Spectroscopy, Paul Scherrer Institut, 5232, Villigen PSI, Switzerland   
$^7$Department of Chemistry, University of British Columbia, Vancouver, BC, V6T 1Z1, Canada    
$^8$Max Planck Institute for the Structure and Dynamics of Matter, 22761 Hamburg, Germany   
$^†$ Present address: Institute for Superconducting and Electronic
Materials, Australian Institute for Innovative Materials, University of Wollongong, North Wollongong, NSW 2500, Australia    
$^‡$ Present address: Department of Physics, Simon Fraser University, Burnaby, Canada V5A 1S6




$^*$ For queries about the supplemental information in this notebook contact [J. Kane Shenton](mailto:john.shenton@mat.ethz.ch).

---





In this notebook we summarise the candidate muon stopping sites in Cr$_2$O$_3$. We identify 4 candidate stopping sites for the bare muon (µ+) and 9 candidate stopping sites for the charge-neutral case (µ0), as listed in Table II of the paper.

Given the $2\times2\times2$ supercell used, and the choice of PAWs described in the paper, the charge-neutral condition has 737 electrons treated explicitly. For the bare muon, 736 electrons are treated explicitly, with a uniform background charge applied to compensate. 


The vasprun.xml files, containing information about both input parameters and output results, corresponding to self-consistent calculations for each candidate site can be found in the mu_plus and mu_zero directories. The sites are labelled as in the paper, Table II. 


We used [ASE](https://wiki.fysik.dtu.dk/ase/index.html) to setup and run the calculations and use [pymatgen](https://pymatgen.org) to parse in the results. 

In [1]:
import numpy as np
import numpy.linalg as la
from pymatgen.io.vasp import Vasprun


In [2]:
def run_summary(run):
    # Check that the run has converged:
    print("Run has converged: {}\n".format(run.converged))

    # Get total energy:
    toten = run.final_energy
    print('TOTEN = {0:16.8f} eV\n'.format(toten))

    # check max force:
    forces = run.ionic_steps[-1]['forces']
    force_magnitudes = la.norm(forces, axis=1)

    print('Max force = {0:6.2f} meV/Å\n'.format(force_magnitudes.max() *1e3))

    #print k-point information
    print(run.kpoints)
    
    # number of electrons in the system:
    nelect = run.parameters['NELECT']
    print('\n# of electrons: ', nelect)

In [3]:
# Base directory that stores the vasprun files
basedir = './'

## Bare muon (µ+)

In [4]:
muon_totens = {}

### Doughnut site

In [5]:

run = Vasprun(basedir + 'mu_plus/D/20190520-1402-vasprun.xml', parse_potcar_file=False)

In [6]:
run_summary(run)

Run has converged: True

TOTEN =    -706.15614751 eV

Max force =   4.78 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  736.0


In [7]:
muon_totens['d'] = run.final_energy

### Bates 0 site

In [8]:

run = Vasprun(basedir + 'mu_plus/B0/20190520-1432-vasprun.xml', parse_potcar_file=False)

In [9]:
run_summary(run)

Run has converged: True

TOTEN =    -705.40275710 eV

Max force =   3.04 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  736.0


In [10]:
muon_totens['b0'] = run.final_energy

### Central site -- exact

In [11]:

run = Vasprun(basedir + 'mu_plus/CE/20190520-1514-vasprun.xml', parse_potcar_file=False)

In [12]:
run_summary(run)

Run has converged: True

TOTEN =    -704.42817080 eV

Max force =   2.85 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  736.0


In [13]:
muon_totens['c0'] = run.final_energy

### Central site -- offset in z

In [14]:

run = Vasprun(basedir + 'mu_plus/CO/20190520-1516-vasprun.xml', parse_potcar_file=False)

In [15]:
run_summary(run)

Run has converged: True

TOTEN =    -704.59156967 eV

Max force =   2.92 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  736.0


In [16]:
muon_totens['c_off'] = run.final_energy

### Site energetics:

In [17]:
min_E = np.array(list(muon_totens.values())).min()
print('Energy above the lowest-energy site')
for key, val in muon_totens.items():
    print("{0:>13} {1:10.2f} meV".format(key, (val - min_E)*1e3))

Energy above the lowest-energy site
            d       0.00 meV
           b0     753.39 meV
           c0    1727.98 meV
        c_off    1564.58 meV


## Charge-neutral muon state (µ0)

In [18]:
Mu_totens = {}

### Doughnut sites:

In [19]:

for d in range(1,6):
    run = Vasprun(basedir + 'mu_zero/D{}/vasprun.xml'.format(d), parse_potcar_file=False)
    print(27*'-')
    print(5*'#'+' Doughnut {} site '.format(d)+5*'#')
    print(27*'-')
    run_summary(run)
    print('\n')
    
    Mu_totens['d{}'.format(d)] = run.final_energy
    
    

---------------------------
##### Doughnut 1 site #####
---------------------------
Run has converged: True

TOTEN =    -696.20710914 eV

Max force =   4.69 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  737.0


---------------------------
##### Doughnut 2 site #####
---------------------------
Run has converged: True

TOTEN =    -696.09115515 eV

Max force =   4.40 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  737.0


---------------------------
##### Doughnut 3 site #####
---------------------------
Run has converged: True

TOTEN =    -695.88706990 eV

Max force =   4.88 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  737.0


---------------------------
##### Doughnut 4 site #####
---------------------------
Run has converged: True

TOTEN =    -695.78789937 eV

Max force =   4.52 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  737.0


---------------------------
##### Doughnut 5 site #####
------------------------

Summarising the doughnut site energies:

In [20]:
minE = min([Mu_totens['d{}'.format(d)] for d in range(1,6)])
for d in range(1,6):
    dE = (Mu_totens['d{}'.format(d)] - minE) * 1000 # convert to meV
    print('dE(D{0}) = {1: 6.1f} meV'.format(d, dE))

dE(D1) =    0.0 meV
dE(D2) =  116.0 meV
dE(D3) =  320.0 meV
dE(D4) =  419.2 meV
dE(D5) =  448.3 meV


### Bates 0 site

In [21]:

run = Vasprun(basedir + 'mu_zero/B0/tetra_20190618-1412-vasprun.xml', parse_potcar_file=False)

In [22]:
run_summary(run)

Run has converged: True

TOTEN =    -695.05014683 eV

Max force =   7.43 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  737.0


In [23]:
Mu_totens['b0'] = run.final_energy

Note that for the Bates 0 site, the force appears to exceed the threshold of 5 meV/Å, however, since this structure was found to be gapless, the Gaussian smearing method was used for the relaxation and the tetrahedron method was used on the final structure, just to get an accurate total energy estimate. See the file: `mu_zero/B0/gauss_vasprun.xml` for the self-consistent run using the Gaussian smearing. 

In [24]:
# Gaussian smearing:
run = Vasprun(basedir + 'mu_zero/B0/gauss_vasprun.xml', parse_potcar_file=False)
run_summary(run)
print(run.incar['ISMEAR'], '-> Gaussian smearing')

Run has converged: True

TOTEN =    -695.05008635 eV

Max force =   4.51 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  737.0
0 -> Gaussian smearing


### Bates 1 site

In [25]:

run = Vasprun(basedir + 'mu_zero/B1/vasprun.xml', parse_potcar_file=False)

In [26]:
run_summary(run)

Run has converged: True

TOTEN =    -695.26189978 eV

Max force =   4.27 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  737.0


In [27]:
Mu_totens['b1'] = run.final_energy

### Central site -- exact

In [28]:

run = Vasprun(basedir + 'mu_zero/CE/20190613-1334-vasprun.xml', parse_potcar_file=False)

In [29]:
run_summary(run)

Run has converged: True

TOTEN =    -695.00066121 eV

Max force =   4.17 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  737.0


In [30]:
Mu_totens['c_e'] = run.final_energy

### Central site -- offset in z

In [31]:

run = Vasprun(basedir + 'mu_zero/CO/20190520-1531-vasprun.xml', parse_potcar_file=False)

In [32]:
run_summary(run)

Run has converged: True

TOTEN =    -695.85199714 eV

Max force =   4.46 meV/Å

Kpoints from vasprun.xml
0
Gamma
4 4 4


# of electrons:  737.0


In [33]:
Mu_totens['c_off'] = run.final_energy

In [34]:
min_E = np.array(list(Mu_totens.values())).min()
print('Energy above the lowest-energy site')
for key, val in Mu_totens.items():
    print("{0:>13} {1:10.1f} meV".format(key.upper(), (val - min_E)*1e3))

Energy above the lowest-energy site
           D1        0.0 meV
           D2      116.0 meV
           D3      320.0 meV
           D4      419.2 meV
           D5      448.3 meV
           B0     1157.0 meV
           B1      945.2 meV
          C_E     1206.4 meV
        C_OFF      355.1 meV
